<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version

Python 3.10.12


In [3]:
# install specific version because latest version uses new version of openAI w/c is incompatible with fireworks api
!pip install -q pyautogen===0.1.14 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
import openai
openai.__version__

'0.28.1'

In [5]:
# @title load fireworks API key
#connect to google drive
from google.colab import drive
import json
import os

drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

os.environ["FIREWORKS_API_KEY"] = env['fireworks.ai']['apiKey']

Mounted at /content/drive


In [6]:
import openai

# This is required to make it work for old version of openai < 1
openai.api_base = "https://api.fireworks.ai/inference/v1"
openai.api_key = env['fireworks.ai']['apiKey']

In [17]:
config_list=[
  {
    "model": "accounts/fireworks/models/mistral-7b-instruct-4k",
    "api_type": "open_ai",
    "api_key": env['fireworks.ai']['apiKey']
  },
  # {
  #   "model": "accounts/fireworks/models/mixtral-8x7b-instruct",
  #   "api_type": "open_ai",
  #   "api_key": env['fireworks.ai']['apiKey']
  # }
]

# configuration for autogen's enhanced inference API which is compatible with OpenAI API
llm_config = {
  "seed": 42,  # seed for caching and reproducibility
  "config_list": config_list,  # a list of OpenAI API configurations
  "temperature": 0,  # temperature for sampling,
  "max_tokens": 4096
}

In [18]:
import autogen
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet"""
)
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To get the current date, you can use the `datetime` module in Python. To compare the year-to-date gain for META and TESLA, you can use the `yfinance` library to retrieve stock data and calculate the gain. Here's the code:
```python
import datetime
import yfinance as yf

# Get the current date
today = datetime.date.today()
print("Today's date:", today)

# Retrieve stock data for META and TESLA
meta_data = yf.Ticker("META").history(period="1y")
tesla_data = yf.Ticker("TSLA").history(period="1y")

# Calculate the year-to-date gain for META and TESLA
meta_gain = (meta_data.iloc[-1] - meta_data.iloc[0]) / meta_data.iloc[0] * 100
tesla_gain = (tesla_data.iloc[-1] - tesla_data.iloc[0]) / tesla_data.iloc[0] * 100

# Print the year-to-date gain for META and TESLA
print("Year-to-date gain fo

In [19]:
!python 'coding/stock_gains.py'

Today's date is: 2024-01-04
FB: No timezone found, symbol may be delisted
Traceback (most recent call last):
  File "/content/coding/stock_gains.py", line 16, in <module>
    close_price = data['Close'][-1]
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/series.py", line 978, in __getitem__
    return self._values[key]
IndexError: index -1 is out of bounds for axis 0 with size 0
